In [ ]:
# Install and import packages
!pip install pyGPGO

from pyGPGO.covfunc import matern32
from pyGPGO.acquisition import Acquisition
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.GPGO import GPGO

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time

In [ ]:
# Connect to Google Colab
from google.colab import drive
from google.colab import output

# This will prompt for authorization to access your Google Drive from Colab.
drive.mount('/content/drive', force_remount=True)

# After mounting, you can navigate to a specific folder using the usual UNIX cd command.
# Replace 'your_folder_path' with the actual path of your folder inside Google Drive.
folder_path = '/content/drive/MyDrive/Colab Notebooks/Truss_project'
package_path = '/content/drive/MyDrive/Colab Notebooks/Truss_project/pyJive'
truss_path = '/content/drive/MyDrive/Colab Notebooks/Truss_project/truss_bridge'

# Support third party widgets
output.enable_custom_widget_manager()

Mounted at /content/drive


In [ ]:
# importing ipympl and importing it (needs custom widgets enabled, that is why it is import seperatly)
!pip install ipympl

import ipympl

In [ ]:
# Manage standard path to the location of the geometry module python file
%cd "$package_path"

# Import geometry module
from geometrymodule import geometry

/content/drive/MyDrive/Colab Notebooks/Truss_project/pyJive


In [ ]:
# Inititalize the geometry package
geom = geometry(package_path, truss_path)

# Defining governing parameters + setup for storage of tested variables
max_iterations = 1500
init_iterations = 20
vars = np.zeros((max_iterations + init_iterations, 19))
best_vars = np.zeros((max_iterations + init_iterations, 19))
best_score = -100000

it = 0

In [ ]:
# Loss function
def f(x1,x2,x3,x4,x5,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14):
    geom.update_nodes([x1,x2,x3,x4,x5])
    geom.update_areas([a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14])
    geom.compute_mass()
    frequency = geom.run_FEM(returns=True, prints=True)
    threshold = geom.FrequencyRequirements

    global it
    vars[it] = [x1,x2,x3,x4,x5,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14]

    mass_score = geom.mass
    frequency_score = 0
    check_score = 0
    for i in range(3):
        if frequency[i] < threshold[i]:
            frequency_score += (threshold[i] - frequency[i])**2 * 10
            check_score += 1000
        else:
            frequency_score += (threshold[i] - frequency[i])**2 * 2

    score = -mass_score - frequency_score - check_score
    global best_score
    global b_vars

    if score > best_score:
        b_vars = [x1,x2,x3,x4,x5,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14]
        best_score = score

    best_vars[it] = b_vars

    it += 1

    return -mass_score - frequency_score - check_score

# Storing the starting time
start_time = time.time()

# Setup the covariance matrix
cov = matern32()

# Setup the gaussian process
gp = GaussianProcess(cov)

# Setup the acquistion function
acq = Acquisition(mode='ExpectedImprovement')

#        Constraints for each parameter         "Optimal" solution
param = {'x1': ('cont', [0.6, 2.2]),            # 0.98
         'x2': ('cont', [0.6, 2.2]),            # 1.38
         'x3': ('cont', [0.6, 2.2]),            # 1.56
         'x4': ('cont', [0.6, 2.2]),            # 1.69
         'x5': ('cont', [0.6, 2.2]),            # 1.76
         'a1': ('cont', [0.00001, 0.001]),      # 0.00030
         'a2': ('cont', [0.00001, 0.001]),      # 0.00010
         'a3': ('cont', [0.00001, 0.001]),      # 0.00010
         'a4': ('cont', [0.00001, 0.001]),      # 0.00026
         'a5': ('cont', [0.00001, 0.001]),      # 0.00012
         'a6': ('cont', [0.00001, 0.001]),      # 0.00011
         'a7': ('cont', [0.00001, 0.001]),      # 0.00023
         'a8': ('cont', [0.00001, 0.001]),      # 0.00015
         'a9': ('cont', [0.00001, 0.001]),      # 0.00015
         'a10': ('cont', [0.00001, 0.001]),     # 0.00023
         'a11': ('cont', [0.00001, 0.001]),     # 0.00012
         'a12': ('cont', [0.00001, 0.001]),     # 0.00012
         'a13': ('cont', [0.00001, 0.001]),     # 0.00025
         'a14': ('cont', [0.00001, 0.001]),     # 0.00010
         }

# Define random seed
np.random.seed(50)

# Initialize the bayesian optimizer
gpgo = GPGO(gp, acq, f, param)

# Run the bayesian optimizer
gpgo.run(max_iter=max_iterations, init_evals=init_iterations)

# Compute total run time
run_time = time.time() - start_time

Streaming output truncated to the last 5000 lines.
 1.00000000e-05 1.00000000e-05 1.00000000e-03 1.00000000e-05
 1.00000000e-03 1.00000000e-05 1.00000000e-03]. 	  -36794.58805140693 	 -1909.0958141446986
Smallest three natural frequencies: [17.127494   55.01328261 88.7429208 ] Hz
131    	 [1.12575905e+00 1.19312084e+00 1.83770651e+00 1.38926346e+00
 1.78621735e+00 6.98085499e-04 4.22939955e-04 1.04522589e-04
 4.53707228e-04 2.58056640e-04 6.55667374e-04 3.21768677e-04
 5.82271104e-04 5.30777163e-04 6.98680985e-04 3.27879460e-04
 4.73443448e-04 8.50052603e-04 8.11752363e-04]. 	  -3649.1712095712655 	 -1909.0958141446986
Smallest three natural frequencies: [ 2.2944722  11.09116866 17.72854279] Hz
132    	 [1.04818031e+00 1.29123734e+00 1.79099070e+00 1.00202498e+00
 1.52037233e+00 2.21721114e-04 1.44840524e-04 1.00000000e-05
 1.00000000e-05 1.00000000e-05 1.00000000e-05 1.00000000e-03
 1.00000000e-05 1.00000000e-05 1.00000000e-05 1.00000000e-03
 1.00000000e-05 8.90469501e-04 1.00000000e-

KeyboardInterrupt: 

In [ ]:
# Acquire optimal result from optimizer
x1, x2, x3, x4, x5 = (gpgo.getResult()[0].get(key) for key in ['x1', 'x2', 'x3', 'x4', 'x5'])
a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14  = (gpgo.getResult()[0].get(key) for key in ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14'])

# Update geometry
geom.update_nodes((x1, x2, x3, x4, x5))
geom.update_areas((a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14))

# Print summary
print(f"The best score found by the optimizer is {-gpgo.getResult()[1]:.2f}")
geom.print_mass()
print(f"The model took {run_time:.2f} seconds to run")
geom.run_FEM(prints=True)
geom.plot_structure()

In [ ]:
#Plotting the objective function
best_eval = gpgo.history
iterations = np.linspace(0, max_iterations, max_iterations+1)
plt.figure()
plt.plot(iterations, [-x for x in best_eval])
plt.grid()
plt.title("Loss function")
plt.xlabel("Iterations (-)")
plt.ylabel("Score (-)")
plt.show()

In [ ]:
# Displaying optimal variables
df = pd.DataFrame(gpgo.getResult()[0], index=[0]).round(5)
df

In [ ]:
# Setup for animation
x_locs = geom.nodes["x"]
y_locs = geom.nodes["y"]
node_map = geom.members[["Node 1", "Node 2"]]
x_map = node_map.replace(np.linspace(0, 19, 20), x_locs).to_numpy().T
y_list1 = np.zeros((2, 37))
y_list2 = np.zeros((2, 37))

# Looping through each iteration
for i in range(len(vars)):
    # Storing each iteration to y_list1
    y_locs[[2, 4, 6, 8, 10, 12, 14, 16, 18]] = [vars[i][0], vars[i][1], vars[i][2], vars[i][3], vars[i][4], vars[i][3], vars[i][2], vars[i][1], vars[i][0]]
    y_map1 = node_map.replace(np.linspace(0, 19, 20), y_locs)
    y_list1 = np.append(y_list1, y_map1.T, axis=0)

    # Storing the best iteration to y_list2
    y_locs[[2, 4, 6, 8, 10, 12, 14, 16, 18]] = [best_vars[i][0], best_vars[i][1], best_vars[i][2], best_vars[i][3], best_vars[i][4], best_vars[i][3], best_vars[i][2], best_vars[i][1], best_vars[i][0]]
    y_map2 = node_map.replace(np.linspace(0, 19, 20), y_locs)
    y_list2 = np.append(y_list2, y_map2.T, axis=0)

In [ ]:
from matplotlib.animation import FuncAnimation
%matplotlib widget

# Initialize the animiation
fig, ax = plt.subplots(2, 1, figsize=(10, 3))
ax[0].set_ylim(0, 2.5)
ax[0].set_aspect("equal")
ax[1].set_ylim(0, 2.5)
ax[1].set_aspect("equal")

line1 = ax[0].plot(x_map, y_list1[2:4], "c-")
line2 = ax[1].plot(x_map, y_list2[2:4], "c-")

# Construct animate function (defines what needs to be updated with each frame)
def animate(i):
    ax[0].set_title(f"iteration {i+1}")
    for j in range(37):
      line1[j].set_ydata(y_list1[2*(i+1):2*(i+2), j])
      line2[j].set_ydata(y_list2[2*(i+1):2*(i+2), j])

    return line1, line2

# Create animation
ani = FuncAnimation(fig=fig, func=animate, frames=(init_iterations + max_iterations), interval=50).save(f"{max_iterations}_iteration_animation.mp4", dpi=600)
plt.show()